In [121]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import pandas as pd
import cv2
import argparse
import mediapipe as mdp
import copy
import os
import csv

In [122]:
#GETS THE REQUIRED ARGUMENTS IN CLI 

def get_args():
    parser=argparse.ArgumentParser()

    parser.add_argument("--device", type= int,default=0)
    parser.add_argument("--staticimage", action="store_true")
    parser.add_argument("--width",help="Maximun width", type= int,default=960)
    parser.add_argument("--height",help="Maximun height", type= int,default=540)

    parser.add_argument("--minDetection" ,help="Minimum confidence for detection",type=float,default=0.6)
    parser.add_argument("--minTracking" ,help="Minimum confidenxce for tracking",type=float,default=0.8)

    args = parser.parse_args()

    return args


In [123]:
def load_models(use_static_image_mode,min_detection,min_tracking):

#LOAD MODEL TO DETECT LANDMARKS IN HANDS
    mdp_hands=mdp.solutions.hands
    hands=mdp_hands.Hands(
        static_image_mode=use_static_image_mode,
        max_num_hands=2,
        min_detection_confidence=min_detection,
        min_tracking_confidence=min_tracking
        )
    return hands

In [124]:
def camera_input(cam_device,cam_width,cam_height):

#POPERLY INITIALIZE CAMERA FOR WORKING
    cam=cv2.VideoCapture(cam_device)
    cam.set(cv2.CAP_PROP_FRAME_WIDTH,cam_width)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT,cam_height)
    print(cam)
    cam.release()
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return cam



In [125]:
# A FUNCTION OT SELECT MODE AND TO SET THE NUMBER TO A VALUE FOR TRAINING
# Probaqbly wont be used but whatever

def select_mode(key,number):
    number=-1
    if 48<=key<=57:
        number=key-48
    if key==110:
        mode=0
    if key==107:
        mode=1
    return number,mode


In [138]:
# NOW TO FIND THE EXACT COORDINATES IN THE SCREEN AS THE IMAGES ARE NORMALIZED RIGHT NPW

def screen_coordinates(normalized_coodr,image):
    img_height,img_width,_=image.shape

    screen_coordinate=[]

    for _,landmark in enumerate(normalized_coodr.landmark):
        screen_x=int(landmark.x*img_width)
        screen_y=int(landmark.y*img_height)
        #NO NEED FOR LANDMARK Z AND ALSO IMAGE.SHAPE[2] PROVIDES COLOR INDEXES LIKE RGB SO 3
        screen_coordinate.append([screen_x,screen_y])
        cv2.circle(image, (screen_x, screen_y), 5, (0, 255, 0), -1)

    return screen_coordinate

In [127]:
#NORMALIZATION SO THAT WE CAN DO SOMETHING EVEN WHEN THE HAND IS MOVED AND IT STILL WORKS ON THE HAND GEUSTURE
#THE NORMALIZED POINTS MAKES IT SO THA WE NEED LESSER DATA AND SIMPLER MODEL TO TRAIN AND DETECT

def normalized_values(screen_coordinate):

    temp_landmark=copy.deepcopy(screen_coordinate)
    base_x,base_y=0,0

    print(type(temp_landmark))

    for i, pixel_val in enumerate(temp_landmark):

        if i==0:
            base_x,base_y=screen_coordinate[0][0],screen_coordinate[0][1]
        
        temp_landmark[i][0]=pixel_val[0]-base_x
        temp_landmark[i][1]=pixel_val[1]-base_y

    temp_landmark=np.array(temp_landmark).flatten()
    max_val=max(list(map(abs,temp_landmark)))

    def normalize_(val):
        return val/max_val
    
    temp_landmark=list(map(normalize_,temp_landmark))

    return temp_landmark
    

        

In [128]:
# NOW TO STORE THE DATAS IN A CSV FILE 
def store_data(number,normalized_one):
    csv_path=r"datas/pre_processed.csv"
    
    with open(csv_path,'a',newline="") as send:
        store=csv.writer(send)
        store.writerow([number,*normalized_one])
    
    return


In [129]:
def image_path(path=''):
    paths=[]
    path_list=[]
    pathx=[]
    if not path:
        path=r"datas\photos_hands"
    all_files=list(map(int,os.listdir(path)))

    for i in range(len(all_files)):
        paths.append(os.path.join(path,str(all_files[i])))
    for path in paths:
        path_list=[]
        for i in os.listdir(path):
            if i.endswith('.png'):
                path_list.append(os.path.join(path,i))
        pathx.append(list(path_list))
        
    return pathx,all_files

image_path()


([['datas\\photos_hands\\1\\image copy.png',
   'datas\\photos_hands\\1\\image.png'],
  ['datas\\photos_hands\\2\\image.png']],
 [1, 2])

In [130]:
image_path()


([['datas\\photos_hands\\1\\image copy.png',
   'datas\\photos_hands\\1\\image.png'],
  ['datas\\photos_hands\\2\\image.png']],
 [1, 2])

In [131]:
def draw_landmarks(image, landmark_point):
    if len(landmark_point) > 0:
        # Thumb
        cv2.line(image, tuple(landmark_point[2]), tuple(landmark_point[3]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[2]), tuple(landmark_point[3]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[3]), tuple(landmark_point[4]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[3]), tuple(landmark_point[4]),
                (255, 255, 255), 2)

        # Index finger
        cv2.line(image, tuple(landmark_point[5]), tuple(landmark_point[6]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[5]), tuple(landmark_point[6]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[6]), tuple(landmark_point[7]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[6]), tuple(landmark_point[7]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[7]), tuple(landmark_point[8]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[7]), tuple(landmark_point[8]),
                (255, 255, 255), 2)

        # Middle finger
        cv2.line(image, tuple(landmark_point[9]), tuple(landmark_point[10]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[9]), tuple(landmark_point[10]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[10]), tuple(landmark_point[11]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[10]), tuple(landmark_point[11]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[11]), tuple(landmark_point[12]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[11]), tuple(landmark_point[12]),
                (255, 255, 255), 2)

        # Ring finger
        cv2.line(image, tuple(landmark_point[13]), tuple(landmark_point[14]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[13]), tuple(landmark_point[14]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[14]), tuple(landmark_point[15]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[14]), tuple(landmark_point[15]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[15]), tuple(landmark_point[16]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[15]), tuple(landmark_point[16]),
                (255, 255, 255), 2)

        # Little finger
        cv2.line(image, tuple(landmark_point[17]), tuple(landmark_point[18]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[17]), tuple(landmark_point[18]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[18]), tuple(landmark_point[19]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[18]), tuple(landmark_point[19]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[19]), tuple(landmark_point[20]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[19]), tuple(landmark_point[20]),
                (255, 255, 255), 2)

        # Palm
        cv2.line(image, tuple(landmark_point[0]), tuple(landmark_point[1]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[0]), tuple(landmark_point[1]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[1]), tuple(landmark_point[2]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[1]), tuple(landmark_point[2]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[2]), tuple(landmark_point[5]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[2]), tuple(landmark_point[5]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[5]), tuple(landmark_point[9]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[5]), tuple(landmark_point[9]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[9]), tuple(landmark_point[13]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[9]), tuple(landmark_point[13]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[13]), tuple(landmark_point[17]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[13]), tuple(landmark_point[17]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[17]), tuple(landmark_point[0]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[17]), tuple(landmark_point[0]),
                (255, 255, 255), 2)

    # Key Points
    for index, landmark in enumerate(landmark_point):
        if index == 0:  # 手首1
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 1:  # 手首2
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 2:  # 親指：付け根
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 3:  # 親指：第1関節
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 4:  # 親指：指先
            cv2.circle(image, (landmark[0], landmark[1]), 8, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 5:  # 人差指：付け根
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 6:  # 人差指：第2関節
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 7:  # 人差指：第1関節
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 8:  # 人差指：指先
            cv2.circle(image, (landmark[0], landmark[1]), 8, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 9:  # 中指：付け根
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 10:  # 中指：第2関節
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 11:  # 中指：第1関節
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 12:  # 中指：指先
            cv2.circle(image, (landmark[0], landmark[1]), 8, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 13:  # 薬指：付け根
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 14:  # 薬指：第2関節
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 15:  # 薬指：第1関節
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 16:  # 薬指：指先
            cv2.circle(image, (landmark[0], landmark[1]), 8, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 17:  # 小指：付け根
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 18:  # 小指：第2関節
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 19:  # 小指：第1関節
            cv2.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 20:  # 小指：指先
            cv2.circle(image, (landmark[0], landmark[1]), 8, (255, 255, 255),
                      -1)
            cv2.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)

    return image


In [132]:
def main():
    
# GET THE CAMERA ARGUMENTS FROM CLI COMMANDS
    cam_device=0
    cam_width=960
    cam_height=540
# GET THE CAMERA ARGUMENTS FROM CLI COMMANDS
    use_static_image_mode=True
    min_detection=0.6
    min_tracking=0.8

    cam=camera_input(cam_device,cam_width,cam_height)
    hands=load_models(use_static_image_mode,min_detection,min_tracking)
    paths,num=image_path()
    idx=0
    print(paths)
    for path in paths:
        for i in path:
            image=cv2.imread(i)
            image = cv2.flip(image, 1)
            # image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            final_img=copy.deepcopy(image)
            
            image.flags.writeable = False
            results = hands.process(image)
            image.flags.writeable = True
    
            screen_coordinate=screen_coordinates(results.multi_hand_landmarks[0],final_img)
           
            normalized_coodr=normalized_values(screen_coordinate)

            draw_landmarks(final_img,screen_coordinate)
    
            cv2.imshow('final',final_img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            store_data(num[idx],normalized_coodr)
          
          
        idx=+1

main()



< cv2.VideoCapture 000002A936A5C630>
[['datas\\photos_hands\\1\\image copy.png', 'datas\\photos_hands\\1\\image.png'], ['datas\\photos_hands\\2\\image.png']]
<class 'list'>


c:\Users\sinju\Documents\HandsDetectSign\handson\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'list'>
<class 'list'>


In [133]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands.
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# Load the image.
image = cv2.imread(r'C:\Users\sinju\Documents\HandsDetectSign\Hand_gesture_detection\datas\photos_hands\1\image copy.png')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Process the image to find hands.
results = hands.process(image_rgb)

# Check if any hands are detected.
if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
        # Draw the landmarks on the image.
        mp.solutions.drawing_utils.draw_landmarks(
            image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
            mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
            mp.solutions.drawing_styles.get_default_hand_connections_style()
        )

# Display the image with landmarks.
cv2.imshow('Hand Landmarks', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [134]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands.
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# Load the image.
image = cv2.imread(r'C:\Users\sinju\Documents\HandsDetectSign\Hand_gesture_detection\datas\photos_hands\1\image copy.png')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
height, width, _ = image.shape

# Process the image to find hands.
results = hands.process(image_rgb)

# Check if any hands are detected.
if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
        # Convert normalized coordinates to pixel coordinates.
        for landmark in hand_landmarks.landmark:
            x_px = int(landmark.x * width)
            y_px = int(landmark.y * height)
            # Draw a circle at each landmark.
            cv2.circle(image, (x_px, y_px), 5, (0, 255, 0), -1)

# Display the image with landmarks.
cv2.imshow('Hand Landmarks', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [135]:
cam_device=0
cam_width=960
cam_height=540



In [136]:
def camerastuff(cam_device,cam_width,cam_height):
    cam=cv2.VideoCapture(cam_device)
    while True:
        cam.set(cv2.CAP_PROP_FRAME_WIDTH,cam_width)
        cam.set(cv2.CAP_PROP_FRAME_HEIGHT,cam_height)
        # Process Key (ESC: end) #################################################
        key = cv2.waitKey(10)
        if key == 27:  # ESC
            break
        

        # Camera capture #####################################################
        ret, image = cam.read()
        if not ret:
            break
        image = cv2.flip(image, 1)  # Mirror display
        debug_image = copy.deepcopy(image)

        cv2.imshow('Hand Gesture Recognition', image)

    cam.release()
    cv2.destroyAllWindows()

camerastuff(cam_device,cam_width,cam_height)

In [137]:
csv_path="../datas/pre_processed.csv"
folder_path="../datas/photos_hands"
all_num=list(map(int,(os.listdir(folder_path))))

all_num

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../datas/photos_hands'

In [ ]:
x=results.multi_hand_landmarks
for i,val in enumerate(x[0].landmark):
    print(val.z)

3.2541322525503347e-07
-0.02299955114722252
-0.030182238668203354
-0.03707556799054146
-0.04414438456296921
-0.006494410336017609
-0.019150547683238983
-0.032826147973537445
-0.04419930651783943
-0.007790715899318457
-0.01980663277208805
-0.03442426025867462
-0.04578622430562973
-0.01307669933885336
-0.028383594006299973
-0.042454879730939865
-0.05228200554847717
-0.02082028239965439
-0.036992985755205154
-0.045942820608615875
-0.05158940702676773


In [ ]:
# Example function to process hand landmarks
def process_hand_landmarks(hand_landmarks: landmark_pb2.NormalizedLandmarkList):
    for idx, landmark in enumerate(hand_landmarks.landmark):
        print(f'Landmark {idx}: x={landmark.x}, y={landmark.y}, z={landmark.z}')

# Example usage with dummy data
example_landmarks = landmark_pb2.NormalizedLandmarkList()
example_landmarks.landmark.add(x=0.5, y=0.5, z=0.5)
example_landmarks.landmark.add(x=0.6, y=0.6, z=0.6)

process_hand_landmarks(example_landmarks)

0
1


In [ ]:
results=main()
output_dict = results.__dict__

# Print the entire dictionary
for key, value in output_dict.items():
    print(f"{key}: {value}")

< cv2.VideoCapture 000001D1AB1DC770>
__doc__: SolutionOutputs(multi_hand_landmarks, multi_hand_world_landmarks, multi_handedness)
__slots__: ()
_fields: ('multi_hand_landmarks', 'multi_hand_world_landmarks', 'multi_handedness')
_field_defaults: {}
__new__: <staticmethod(<function SolutionOutputs.__new__ at 0x000001D1AB3F6700>)>
_make: <classmethod(<function SolutionOutputs._make at 0x000001D1AB1C3D80>)>
_replace: <function SolutionOutputs._replace at 0x000001D1AB1C37E0>
__repr__: <function SolutionOutputs.__repr__ at 0x000001D190311620>
_asdict: <function SolutionOutputs._asdict at 0x000001D190313F60>
__getnewargs__: <function SolutionOutputs.__getnewargs__ at 0x000001D1AB1F9260>
__match_args__: ('multi_hand_landmarks', 'multi_hand_world_landmarks', 'multi_handedness')
multi_hand_landmarks: [landmark {
  x: 0.603820086
  y: 0.531883359
  z: 3.25413225e-007
}
landmark {
  x: 0.579289913
  y: 0.422052443
  z: -0.0229995511
}
landmark {
  x: 0.537265062
  y: 0.348211676
  z: -0.0301822387